<a href="https://colab.research.google.com/github/ltklinh1610/KTLT-ECO-GROUP/blob/main/Cu%E1%BB%91i%20K%C3%AC/CuoiKi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cuối Kì

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer # Giữ lại để mô phỏng bước Impute trong báo cáo

# KIỂM TRA DỮ LIỆU

In [15]:
import pandas as pd

df = pd.read_csv('Loan_default.csv')

print("=== THÔNG TIN TỔNG QUAN VÀ THỐNG KÊ MÔ TẢ (Describe All) ===")
print(df.describe())

print(f"Kích thước dữ liệu gốc: {df.shape}")
print("5 dòng đầu tiên:\n")
print(df.head(5))

print("\nKiểm tra Non-Null Count và Kiểu dữ liệu (Dữ liệu gốc):\n")
df.info()

=== THÔNG TIN TỔNG QUAN VÀ THỐNG KÊ MÔ TẢ (Describe All) ===
                 Age         Income     LoanAmount    CreditScore  \
count  255347.000000  255347.000000  255347.000000  255347.000000   
mean       43.498306   82499.304597  127578.865512     574.264346   
std        14.990258   38963.013729   70840.706142     158.903867   
min        18.000000   15000.000000    5000.000000     300.000000   
25%        31.000000   48825.500000   66156.000000     437.000000   
50%        43.000000   82466.000000  127556.000000     574.000000   
75%        56.000000  116219.000000  188985.000000     712.000000   
max        69.000000  149999.000000  249999.000000     849.000000   

       MonthsEmployed  NumCreditLines   InterestRate       LoanTerm  \
count   255347.000000   255347.000000  255347.000000  255347.000000   
mean        59.541976        2.501036      13.492773      36.025894   
std         34.643376        1.117018       6.636443      16.969330   
min          0.000000        1.00

# Làm Sạch dữ liệu

In [ ]:
CORE_COLUMNS = ['Default', 'Age', 'Income', 'LoanAmount', 'InterestRate', 'LoanTerm', 'EmploymentType', 'Education']
df_filtered = df[CORE_COLUMNS].copy()

# LẤY MẪU NGẪU NHIÊN (3% - 7661 quan sát)
sample_frac = 0.03
df_sample = df_filtered.sample(frac=sample_frac, random_state=42).copy()

# ĐỔI TÊN CỘT SANG TIẾNG VIỆT
df_sample.columns = ['Khả năng vỡ nợ', 'Tuổi', 'Thu nhập', 'Khoản vay', 'Lãi suất vay', 'Thời hạn vay', 'Nghề nghiệp', 'Trình độ học vấn']

# KIỂM TRA VÀ MÔ PHỎNG IMAPUTE (Data Cleaning)
# Dùng SimpleImputer để mô phỏng "Average/Most frequent"
numerical_cols = ['Tuổi', 'Thu nhập', 'Khoản vay', 'Lãi suất vay', 'Thời hạn vay']
categorical_cols = ['Nghề nghiệp', 'Trình độ học vấn']
imputer_numeric = SimpleImputer(strategy='mean')
imputer_categorical = SimpleImputer(strategy='most_frequent')

df_sample[numerical_cols] = imputer_numeric.fit_transform(df_sample[numerical_cols])
df_sample[categorical_cols] = imputer_categorical.fit_transform(df_sample[categorical_cols])

# CHUẨN HÓA BIẾN VÀ MÃ HÓA (Data Transformation)
# Chuẩn hóa nhị phân
df_sample['Trình độ học vấn_Binary'] = df_sample['Trình độ học vấn'].apply(
    lambda x: 1 if x in ['Master\'s', 'PhD'] else 0
)
df_sample['Nghề nghiệp_Binary'] = df_sample['Nghề nghiệp'].apply(
    lambda x: 0 if x == 'Unemployed' else 1
)

# Mã hóa One-Hot cho Phân loại
df_encoded = pd.get_dummies(df_sample,
                           columns=['Nghề nghiệp', 'Trình độ học vấn'],
                           prefix=['NgheNghiep', 'TrinhDo'])

print(f"Tiền xử lý hoàn tất. Kích thước DataFrame cuối cùng: {df_encoded.shape}")
print(df_encoded.describe())
print(df_encoded.head(5))

# BƯỚC 3 PHÂN TÍCH MÔ TẢ

In [ ]:
print("\n=== BƯỚC 3: PHÂN TÍCH MÔ TẢ (EDA) ===")
numerical_cols = ['Tuổi', 'Thu nhập', 'Khoản vay', 'Lãi suất vay', 'Thời hạn vay']

age_bins = [15, 30, 45, 60, 75]
age_labels = ['15-30', '31-45', '46-60', '61-75']
df_sample['Tuổi_Group'] = pd.cut(df_sample['Tuổi'], bins=age_bins, labels=age_labels, right=True, include_lowest=True)

loan_max = df_sample['Khoản vay'].max()
loan_bins = np.arange(5000, 275001, 30000)
df_sample['Khoản vay_Group'] = pd.cut(df_sample['Khoản vay'], bins=loan_bins, include_lowest=True)

income_max = df_sample['Thu nhập'].max()
income_bins = np.arange(15000, 155001, 15000)
df_sample['Thu nhập_Group'] = pd.cut(df_sample['Thu nhập'], bins=income_bins, include_lowest=True)

rate_bins = [0, 5, 10, 15, 20, 25]
rate_labels = ['0.00-5.00', '5.00-10.00', '10.00-15.00', '15.00-20.00', '20.00-25.00']
df_sample['Lãi suất vay_Group'] = pd.cut(df_sample['Lãi suất vay'], bins=rate_bins, labels=rate_labels, right=True, include_lowest=True)

df_sample['Thời hạn vay_Group'] = df_sample['Thời hạn vay'].astype(str)

plt.figure(figsize=(20, 10))
plt.suptitle('Dashboard: Biểu đồ Cột Cố định', fontsize=16)

grouped_cols = ['Tuổi_Group', 'Khoản vay_Group', 'Thu nhập_Group', 'Lãi suất vay_Group', 'Thời hạn vay_Group']
group_plot_positions = [3, 4, 5, 6, 7]

plt.subplot(2, 4, 1)
df_sample['Trình độ học vấn'].value_counts().plot.pie(autopct='%1.1f%%', startangle=90, cmap='viridis')
plt.title('1. Tỉ trọng Trình độ học vấn')
plt.ylabel('')

plt.subplot(2, 4, 2)
df_sample['Nghề nghiệp'].value_counts().plot.pie(autopct='%1.1f%%', startangle=90, cmap='Pastel2')
plt.title('2. Tỉ trọng Tình trạng việc làm')
plt.ylabel('')

for i, col_group in enumerate(grouped_cols):
    plt.subplot(2, 4, group_plot_positions[i])

    # Sử dụng Countplot để tạo Biểu đồ cột
    ax = sns.countplot(data=df_sample, x=col_group, palette='tab10',
                       order=df_sample[col_group].value_counts().index)
    plt.title(f'{group_plot_positions[i]}. Phân phối {col_group.split("_")[0]}')
    plt.xlabel('')
    plt.xticks(rotation=45, ha='right', fontsize=8)

    # Gắn nhãn số lượng lên trên các thanh bar
    for p in ax.patches:
        height = p.get_height()
        if height > 0:
            ax.text(p.get_x() + p.get_width() / 2.,
                    height + 50,
                    '{:1.0f}'.format(height),
                    ha="center", fontsize=8, weight='bold')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


#PHÂN TÍCH ĐIỂM ĐẶC THÙ 
print("\n[Phân tích] Distribution theo Khả năng vỡ nợ (Sử dụng Biểu đồ Cột Stacked):")

for col_group in grouped_cols:
    plt.figure(figsize=(10, 6))

    data_counts = df_sample.groupby([col_group, 'Khả năng vỡ nợ']).size().unstack(fill_value=0)
    data_counts.plot(kind='bar', stacked=True, color=['#1f77b4', '#ff7f0e'], ax=plt.gca())

    plt.title(f'Distribution của {col_group.split("_")[0]} (0: Không vỡ nợ, 1: Vỡ nợ)')
    plt.xlabel(col_group.replace('_', ' '))
    plt.ylabel('Số lượng')
    plt.xticks(rotation=45, ha='right', fontsize=9)
    plt.legend(title='Khả năng vỡ nợ', labels=['0 (Không Vỡ nợ)', '1 (Vỡ nợ)'])
    plt.show()